In [1]:
import findspark

In [2]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F

In [4]:
! ls -l $SPARK_HOME/jars | grep postgresql

-rw-rw-r--. 1 train train   932808 Jul 22 08:42 postgresql-42.2.14.jar


In [5]:
# If you want to connect a rdbms, related driver should be in the classpath

In [5]:
spark = SparkSession.builder \
.appName("JDBC and SQL") \
.master("yarn") \
.enableHiveSupport() \
.getOrCreate()

# Read From Postgresql

In [6]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [7]:
df = (spark.read
.format("jdbc")
.option("url", jdbcUrl)
.option("driver", 'org.postgresql.Driver')
.option("query","select * from books")
.load())

In [8]:
df.limit(5).toPandas()

,id,book_name,isbn,book_id,price,price_currency,rating_count,author_id,publisher_id
0,13,Madam Bovary (Ciltli),6050948752,489127179,25.115735,TRY,5,4098249,46868
1,22,Mai ve Siyah (Eleştirel Basım),9750523533,492625951,25.340000,TRY,17,3066057,63217
2,27,Nutuk,9759914288,9927355,11.481470,TRY,23,9705003,46868
3,34,Devlet,9754734263,395307782,27.999400,TRY,0,8978000,20709


# Write to Postgresql

In [9]:
df.write.jdbc(url=jdbcUrl,
              table="books_spark", 
              mode="overwrite", 
              properties={"driver": 'org.postgresql.Driver'})

## Check write result

In [12]:
(spark.read
.format("jdbc")
.option("url", jdbcUrl)
.option("driver", 'org.postgresql.Driver')
.option("query","select * from books_spark")
.load()).limit(5).toPandas()

,id,book_name,isbn,book_id,price,price_currency,rating_count,author_id,publisher_id
0,13,Madam Bovary (Ciltli),6050948752,489127179,25.115735,TRY,5,4098249,46868
1,22,Mai ve Siyah (Eleştirel Basım),9750523533,492625951,25.340000,TRY,17,3066057,63217
2,27,Nutuk,9759914288,9927355,11.481470,TRY,23,9705003,46868
3,34,Devlet,9754734263,395307782,27.999400,TRY,0,8978000,20709


In [13]:
spark.stop()